In [28]:
#> python3 -m pip install --upgrade tensorflowhttp://localhost:8888/notebooks/Untitled.ipynb?kernel_name=python3#
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
tf.__version__

'2.0.0'

====================CLASSIFICATION

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test,y_test) = fashion_mnist.load_data()

In [4]:
print(X_train_full.shape)
print(X_train_full.dtype)

(60000, 28, 28)
uint8


    1. scale pixel density to 0-1, 
    2. carve out validation set, and trainset

In [5]:
X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

    fashion_MNIST needs manual naming to target values naming

In [6]:
class_names=["T-shirt/top", "Trouser", "Pullover", "Dress", "Fluffy Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
class_names[y_train[0]]

'Fluffy Coat'

       use Sequential API, build a neural network (2-layer MLP)
       Sequential MLP is the simpliest form of ANN , that is a single stack of layers connectioned sequentially

In [7]:
model = keras.models.Sequential([

keras.layers.Flatten(input_shape = [28,28]),
keras.layers.Dense(300, activation = "relu"),
keras.layers.Dense(100, activation = "relu"),
keras.layers.Dense(10, activation = "softmax") #last layer spit out 10 classes
    
])
print(model.layers)
print(model.summary())

[<tensorflow.python.keras.layers.core.Flatten object at 0x0000024744ADE6A0>, <tensorflow.python.keras.layers.core.Dense object at 0x0000024744AC7668>, <tensorflow.python.keras.layers.core.Dense object at 0x0000024744AA0D30>, <tensorflow.python.keras.layers.core.Dense object at 0x0000024744ADEA20>]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_______________

       Note: 266,610 = 300 (nodes) x 784 (features) + 300 (bias terms, b)

In [8]:
# access layers and Weights/Bias

hidden_l1 = model.layers[1]
w1, b1 = hidden_l1.get_weights()
print(w1.shape)
print(b1.shape)
print(w1)

(784, 300)
(300,)
[[-0.04035427  0.01333623  0.06378546 ... -0.00396299 -0.03340212
   0.01419149]
 [-0.01565997 -0.00216973 -0.00887952 ...  0.00159066 -0.0604704
  -0.04055049]
 [-0.03045494  0.05493841  0.04392813 ...  0.00097772  0.0029441
  -0.05589407]
 ...
 [-0.00513101  0.01343355  0.04313413 ...  0.02039172 -0.0312417
  -0.03538526]
 [-0.0262517  -0.00551044  0.03687672 ...  0.03292245 -0.04892159
  -0.04664678]
 [ 0.00406942  0.01297504  0.02957294 ...  0.03105066  0.07419251
  -0.00743509]]


initial weights are random, initial bias are 0s, 
to change this, use kernel_initializer ,bias_initializer via https://keras.io/initializers

____________________

Use compile() to set loss_function, optimizer/learning_algo, and accuracy metrics<BR>
when using SGD (Stochastic Gradient Decsent, tuning learning rate is critical)

In [9]:
model.compile(
    loss= "sparse_categorical_crossentropy", 
    optimizer=keras.optimizers.SGD(lr = 0.01), 
    metrics = ["accuracy"])

history = model.fit(X_train, y_train, epochs = 30, validation_data = (X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 5s 85us/sample - loss: 0.7170 - accuracy: 0.7635 - val_loss: 0.5014 - val_accuracy: 0.8284
Epoch 2/30
55000/55000 [==============================] - 4s 74us/sample - loss: 0.4892 - accuracy: 0.8293 - val_loss: 0.4384 - val_accuracy: 0.8528
Epoch 3/30
55000/55000 [==============================] - 4s 73us/sample - loss: 0.4434 - accuracy: 0.8450 - val_loss: 0.4145 - val_accuracy: 0.8620
Epoch 4/30
55000/55000 [==============================] - 4s 72us/sample - loss: 0.4153 - accuracy: 0.8535 - val_loss: 0.4278 - val_accuracy: 0.8538
Epoch 5/30
55000/55000 [==============================] - 4s 72us/sample - loss: 0.3959 - accuracy: 0.8603 - val_loss: 0.3936 - val_accuracy: 0.8652
Epoch 6/30
55000/55000 [==============================] - 4s 73us/sample - loss: 0.3796 - accuracy: 0.8662 - val_loss: 0.4068 - val_accuracy: 0.8554
Epoch 7/30
55000/55000 [==============================] -

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

<Figure size 800x500 with 1 Axes>

Hierarchy of fine-tuning your hyperparameters
1. learning rate
2. optimizer (and then learning rate)
3. number of layers, 
4. number of nerous per layers
5. batch size

Once validation/accuracy is good enough on the validation set

evaulate the fitted model on test set:

In [11]:
model.evaluate(X_test, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[70.48442838606871, 0.8402]

.predict() and .predict_classes()

the former produces probabilities for each class <br>
the latter produces the class with the highest 

In [30]:
print((model.predict(X_test[:3])).round(3))
y_pred = (model.predict_classes(X_test[:2]))
print(y_pred)
np.array(class_names)[y_pred]


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[9 2]


array(['Ankle boot', 'Pullover'], dtype='<U11')

====================REGRESSION

In [32]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

model_r = keras.models.Sequential([
    keras.layers.Dense(30, activation = "relu", input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])

model_r.compile(loss="mean_squared_error", optimizer = "sgd")
history = model_r.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid))

mse_test = model_r.evaluate(X_test, y_test)
y_pred_r = model_r.predict(X_test[:3])
print(y_pred_r)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 72us/sample - loss: 0.8291 - val_loss: 0.6054
Epoch 2/20
11610/11610 [==============================] - 1s 43us/sample - loss: 0.6085 - val_loss: 0.5064
Epoch 3/20
11610/11610 [==============================] - 0s 43us/sample - loss: 0.4866 - val_loss: 0.5188
Epoch 4/20
11610/11610 [==============================] - 0s 42us/sample - loss: 0.4930 - val_loss: 0.6086
Epoch 5/20
11610/11610 [==============================] - 1s 43us/sample - loss: 0.4423 - val_loss: 0.6877
Epoch 6/20
11610/11610 [==============================] - 0s 43us/sample - loss: 0.4307 - val_loss: 0.8303
Epoch 7/20
11610/11610 [==============================] - 1s 43us/sample - loss: 0.4282 - val_loss: 0.9727
Epoch 8/20
11610/11610 [==============================] - 0s 42us/sample - loss: 0.4434 - val_loss: 1.0082
Epoch 9/20
11610/11610 [==============================] - 1s 43us/sample - loss: 0.4112 - val_l

[[2.1240609]
 [3.2003384]
 [1.3764153]]
